Tuning opportunities
1. Configuration-Based Hyperparameter Tuning
2. Enhanced Data Augmentation
3. Improved Model Architecture
4. Advanced Callbacks
5. Transfer Learning (Often Best Results)
6. Class Imbalance Handling
7. Ensemble Methods
8. Systematic Tuning Strategy

Quick Start Guide:
For best results immediately: Set "backbone": "mobilenet" and "fine_tune": True
For faster training: Set "backbone": "custom_cnn" and "aug_level": "light"
For maximum accuracy: Enable ensemble with "use_ensemble": True
For experimentation: Uncomment Cell 17 to run systematic tuning

In [1]:
# =======================================================================
# Facial Emotion Recognition - Enhanced with 8 Tuning Improvements
# =======================================================================

import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image, UnidentifiedImageError
from pathlib import Path
from datetime import datetime

# TensorFlow and Keras
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"Python executable: {sys.executable}")

import seaborn as sns

# Deep learning libraries
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Flatten, Dropout, Dense, Input, 
    GlobalAveragePooling2D, Conv2D, 
    BatchNormalization, Activation, MaxPooling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, 
    ReduceLROnPlateau, LearningRateScheduler)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight


print("=" * 70)
print("= is device GPU device")
print("=" * 70)

from tensorflow.keras import mixed_precision
import tensorflow as tf

# Enable mixed precision only if a GPU is available
if tf.config.list_physical_devices('GPU'):
    mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision enabled: mixed_float16")
else:
    mixed_precision.set_global_policy('float32')
    print("No GPU detected — using float32")
    
print("Global policy:", mixed_precision.global_policy())


print("=" * 70)
print("✅ All libraries imported successfully")

2025-11-12 02:23:12.180237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762914192.406982      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762914192.477898      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

TensorFlow version: 2.18.0
Python executable: /usr/bin/python3
= is device GPU device
Mixed precision enabled: mixed_float16
Global policy: <DTypePolicy "mixed_float16">
✅ All libraries imported successfully


In [2]:
# =======================================================================
# Environment Detection
# =======================================================================

def is_on_kaggle():
    """Detect if running on Kaggle."""
    return os.path.exists('/kaggle/input')

def get_data_path():
    """Detect environment and return appropriate data path."""
    if is_on_kaggle():
        print("🌐 Running on Kaggle")
        import kagglehub
        image_path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")
        folder_path = os.path.join(image_path, "images")
    else:
        print("💻 Running on local machine")
        folder_path = "data/images/"
    
    return folder_path

folder_path = get_data_path()
print(f"Data folder path: {folder_path}")

🌐 Running on Kaggle
Data folder path: /kaggle/input/face-expression-recognition-dataset/images


In [3]:
# =======================================================================
# Enhanced Configuration with Tuning Options
# =======================================================================

cfg = {
    # Model Architecture
    "backbone": "mobilenet",  # Options: "custom_cnn", "mobilenet", "efficientnet"
    
    # Image Parameters
    "picture_size": 48,  # Try: 64, 96 for more detail
    "color_mode": "grayscale",  # "grayscale" or "rgb" (rgb required for transfer learning)
    
    # Training Parameters
    "batch_size": 64,  # Reduced from 128 for better gradient estimates
    "epochs": 5,  # Increased from 30
    "learning_rate": 0.0001,  # Adjusted learning rate
    "dropout_rate": 0.25,  # Increased from 0.25
     
    # Dense layers
    "dense_units": [512],  # Added an extra dense layer (Improvement #6)
    
    # Data Augmentation (Improvement #2)
    "aug_level": "light",  # Options: "none", "light", "strong", "aggressive"
    
    # Advanced Training
    "precision": "mixed",  # "float32" or "mixed"
    "fine_tune": True,  # Enable fine-tuning for transfer learning
    "fine_tune_epochs": 0,
    
    # Optimizer
    "optimizer": "adam",  # Options: "adam", "adamw", "sgd"
    "weight_decay": 1e-4,  # For AdamW
    
    # Learning Rate Schedule
    "use_lr_schedule": True,  # Cosine annealing (Improvement #5)
    
    # Class Weights (Improvement #3)
    "use_class_weights": True,
    
    # Ensemble (Improvement #7)
    "use_ensemble": False,  # Set True to train multiple models
    "n_ensemble_models": 3,
    
    # Model Parameters
    "no_of_classes": 7,
}

print("Configuration loaded:")
for key, value in cfg.items():
    print(f"  {key}: {value}")

# Extract commonly used values
picture_size = cfg["picture_size"]
batch_size = cfg["batch_size"]
epochs = cfg["epochs"]
learning_rate = cfg["learning_rate"]
no_of_classes = cfg["no_of_classes"]
dropout_rate = cfg["dropout_rate"]
dense_units = cfg["dense_units"]

Configuration loaded:
  backbone: mobilenet
  picture_size: 48
  color_mode: grayscale
  batch_size: 64
  epochs: 5
  learning_rate: 0.0001
  dropout_rate: 0.25
  dense_units: [512]
  aug_level: light
  precision: mixed
  fine_tune: True
  fine_tune_epochs: 0
  optimizer: adam
  weight_decay: 0.0001
  use_lr_schedule: True
  use_class_weights: True
  use_ensemble: False
  n_ensemble_models: 3
  no_of_classes: 7


In [4]:
# near your cfg definition (apply before building generators)
cfg.update({
    "picture_size": 96,
    "color_mode": "rgb",
    "head_epochs": 10,
    "fine_tune": True,
    "fine_tune_epochs": 20,
    "fine_tune_unfreeze_layers": 30,
})
# rebind frequently used variables
picture_size = cfg["picture_size"]
color_mode = cfg["color_mode"]

print("Configuration loaded:")
for key, value in cfg.items():
    print(f"  {key}: {value}")

Configuration loaded:
  backbone: mobilenet
  picture_size: 96
  color_mode: rgb
  batch_size: 64
  epochs: 5
  learning_rate: 0.0001
  dropout_rate: 0.25
  dense_units: [512]
  aug_level: light
  precision: mixed
  fine_tune: True
  fine_tune_epochs: 20
  optimizer: adam
  weight_decay: 0.0001
  use_lr_schedule: True
  use_class_weights: True
  use_ensemble: False
  n_ensemble_models: 3
  no_of_classes: 7
  head_epochs: 10
  fine_tune_unfreeze_layers: 30


In [5]:
# =======================================================================
# Enhanced Data Augmentation
# =======================================================================

aug_map = {
    "none": dict(rescale=1./255),
    
    "light": dict(
        rescale=1./255,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True
    ),
    
    "strong": dict(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.2,
        brightness_range=[0.7, 1.3],
        horizontal_flip=True,
        fill_mode='nearest'
    ),
    
    "aggressive": dict(
        rescale=1./255,
        rotation_range=30,
        width_shift_range=0.25,
        height_shift_range=0.25,
        shear_range=0.2,
        zoom_range=0.25,
        brightness_range=[0.6, 1.4],
        horizontal_flip=True,
        channel_shift_range=0.2,
        fill_mode='nearest'
    )
}

# Create data generators
datagen_train = ImageDataGenerator(**aug_map[cfg["aug_level"]])
datagen_validation = ImageDataGenerator(**aug_map[cfg["aug_level"]])

print(f"✅ Using '{cfg['aug_level']}' augmentation level for training data")
print(f"✅ Using '{cfg['aug_level']}' augmentation level for validation data")

✅ Using 'light' augmentation level for training data
✅ Using 'light' augmentation level for validation data


In [6]:
# =======================================================================
# Load Training and Validation Data
# =======================================================================

# Determine color mode
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Prepare augmentation for train (remove rescale if present)
train_aug = aug_map[cfg["aug_level"]].copy()
train_aug.pop("rescale", None)
train_aug.update({"preprocessing_function": preprocess_input})
datagen_train = ImageDataGenerator(**train_aug)

# Validation: only preprocessing (no augmentation)
datagen_validation = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create flows (must use the same picture_size and color_mode)
train_set = datagen_train.flow_from_directory(
    os.path.join(folder_path, "train"),
    target_size=(picture_size, picture_size),
    color_mode=color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
)

validation_set = datagen_validation.flow_from_directory(
    os.path.join(folder_path, "validation"),
    target_size=(picture_size, picture_size),
    color_mode=color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
)

print("Train image shape:", train_set.image_shape)
print("Val image shape:  ", validation_set.image_shape)

print(f"\n✅ Data loaded successfully")
print (f"Colour Mode: {color_mode}")
print(f"Training samples: {train_set.n}")
print(f"Validation samples: {validation_set.n}")
print(f"Class indices: {train_set.class_indices}")

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Train image shape: (96, 96, 3)
Val image shape:   (96, 96, 3)

✅ Data loaded successfully
Colour Mode: rgb
Training samples: 28821
Validation samples: 7066
Class indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [7]:
# =======================================================================
# Calculate Class Weights for Imbalanced Data
# =======================================================================

class_weight_dict = None

if cfg.get("use_class_weights", False):
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_set.classes),
        y=train_set.classes
    )
    class_weight_dict = dict(enumerate(class_weights))
    
    print("\n✅ Class weights calculated:")
    for emotion, weight in zip(class_labels, class_weights):
        print(f"  {emotion:>10}: {weight:.3f}")
else:
    print("\n⚠️  Class weights disabled")


✅ Class weights calculated:
       Angry: 1.031
     Disgust: 9.443
        Fear: 1.003
       Happy: 0.575
     Neutral: 0.826
         Sad: 0.834
    Surprise: 1.285


In [8]:
def build_transfer_learning_model(cfg):
    """Build and compile a MobileNetV2 transfer-learning model.
       Returns (model, base_model).
    """

    picture_size = cfg["picture_size"]
    color_mode = cfg["color_mode"]
    batch_size = cfg["batch_size"]

    channels = 1 if cfg["color_mode"] == "grayscale" else 3
    input_shape = (picture_size, picture_size, channels)

    # Input
    inp = Input(shape=input_shape)

    # If grayscale, convert to RGB for ImageNet backbone
    if channels == 1:
        x = tf.keras.layers.Lambda(lambda z: tf.image.grayscale_to_rgb(z))(inp)
    else:
        x = inp

    # Pre-trained backbone
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(picture_size, picture_size, 3), pooling=None)
    # Attach to our RGB input
    x = base_model(x, training=False)

    # Classification head
    x = GlobalAveragePooling2D()(x)
    x = Dropout(cfg["dropout_rate"])(x)
    for units in cfg["dense_units"]:
        x = Dense(units, activation='relu')(x)
        x = Dropout(cfg["dropout_rate"])(x)
    out = Dense(no_of_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs=inp, outputs=out)

    # Freeze base model initially
    if not cfg.get("fine_tune", False):
        base_model.trainable = False
    else:
        # Start by freezing; we'll unfreeze later for fine-tuning if desired
        base_model.trainable = False

    # Optimizer selection (same logic as your custom CNN)
    optimizer_name = cfg.get("optimizer", "adam")
    if optimizer_name == "adam":
        opt = Adam(learning_rate=cfg["learning_rate"])
    elif optimizer_name == "adamw":
        opt = tf.keras.optimizers.AdamW(
            learning_rate=cfg["learning_rate"],
            weight_decay=cfg.get("weight_decay", 1e-4)
        )
    elif optimizer_name == "sgd":
        opt = tf.keras.optimizers.SGD(
            learning_rate=cfg["learning_rate"],
            momentum=0.9,
            nesterov=True
        )

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model, base_model

In [9]:
# =======================================================================
# Enhanced Model Building Functions
# =======================================================================


def build_improved_cnn(cfg):
    """Enhanced CNN with better architecture."""
    
    
    #color_mode = cfg.get("color_mode", "grayscale")
    channels = 1 if color_mode == "grayscale" else 3
    input_shape = (picture_size, picture_size, channels)
    
    print(f"Building Improved CNN with input shape: {input_shape}")
    
    model = Sequential()
    
    # Block 1 - Double Conv
    model.add(Conv2D(filters=32, 
                        kernel_size=(3, 3), 
                        padding='same', 
                        input_shape=input_shape,
                        name='conv2d_1'))

    # Block 2 - Double Conv
    model.add(Conv2D(filters=128, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_2'))
    model.add(BatchNormalization(name='bn_2'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_2'))
    model.add(Dropout(dropout_rate,name='dropout_2'))
    
    # Block 3 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(5, 5), 
                      padding='same',
                      name='conv2d_3'))
    model.add(BatchNormalization(name='bn_3'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_3'))
    model.add(Dropout(dropout_rate,name='dropout_3'))
    
    # Block 4 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_4'))
    model.add(BatchNormalization(name='bn_4'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_4'))
    model.add(Dropout(dropout_rate,name='dropout_4'))

    
    # Block 5 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_5'))
    model.add(BatchNormalization(name='bn_5'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_5'))
    model.add(Dropout(dropout_rate,name='dropout_5'))
    
    """
        # Block 6 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_6'))
    model.add(BatchNormalization(name='bn_6'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_6'))
    model.add(Dropout(dropout_rate,name='dropout_6'))
    """
    
    
    # Output layer
    # Add dense layers before output
    model.add(Flatten())
    for idx, units in enumerate(dense_units):
        print(f"Adding dense layer with {units} units")
        model.add(Dense(units, name=f"dense_{idx+1}"))
        model.add(BatchNormalization(name=f"bn_dense_{idx+1}"))
        model.add(Activation('relu', name=f"act_dense_{idx+1}"))
        model.add(Dropout(dropout_rate, name=f"dropout_dense_{idx+1}"))
    model.add(Dense(no_of_classes, activation='softmax', dtype='float32', name='output_layer'))

   
    # Compile with optimizer choice
    optimizer_name = cfg.get("optimizer", "adam")
    if optimizer_name == "adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer_name == "adamw":
        opt = tf.keras.optimizers.AdamW(
            learning_rate=learning_rate,
            weight_decay=cfg.get("weight_decay", 1e-4)
        )
    elif optimizer_name == "sgd":
        opt = tf.keras.optimizers.SGD(
            learning_rate=learning_rate,
            momentum=0.9,
            nesterov=True
        )
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    
    return model, None

# Build model based on configuration
print(f"\n{'='*70}")
print(f"Building model with '{cfg['backbone']}' backbone...")
print(f"{'='*70}")


Building model with 'mobilenet' backbone...


In [10]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Prepare augmentation dict for train (remove rescale if present)
train_aug = aug_map[cfg["aug_level"]].copy()
train_aug.pop("rescale", None)          # MobileNet uses preprocess_input
train_aug.update({"preprocessing_function": preprocess_input})
datagen_train = ImageDataGenerator(**train_aug)

# Validation: only preprocessing (no augmentation)
datagen_validation = ImageDataGenerator(preprocessing_function=preprocess_input)

# Prepare augmentation dict for validation
if cfg["backbone"] == "mobilenet":
    train_preproc = dict(preprocessing_function=preprocess_input)
    val_preproc = dict(preprocessing_function=preprocess_input)
else:
    train_preproc = val_preproc = {}

aug = aug_map[cfg["aug_level"]].copy()
aug.update(train_preproc)
datagen_train = ImageDataGenerator(**aug)

# For validation: do NOT use augmentation, only preprocessing/rescale
val_aug = {"preprocessing_function": preprocess_input} if cfg["backbone"] == "mobilenet" else {"rescale": 1./255}
datagen_validation = ImageDataGenerator(**val_aug)



In [11]:

# Replace the model building section with:
if cfg["backbone"] == "custom_cnn":
    model, base_model = build_improved_cnn(cfg)
elif cfg["backbone"] == "mobilenet":
    model, base_model = build_transfer_learning_model(cfg)
else:
    raise ValueError(f"Unsupported backbone: {cfg['backbone']}")

print(f"\n✅ Model built successfully")
model.summary()

I0000 00:00:1762914234.121603      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1762914234.122328      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

✅ Model built successfully


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_96             │ (None, 3, 3, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_1 (Cast)                   │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,917,447 (11.13 MB)

 Trainable params: 659,463 (2.52 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [12]:
# =======================================================================
# Advanced Training Callbacks
# =======================================================================

# Checkpoint
checkpoint = ModelCheckpoint('best_model.keras',
                                    monitor='val_accuracy',
                                    save_best_only=True,
                                    mode='max',
                                    verbose=1
                                )

# Early stopping with more patience
early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=5,  # Increased from 10
                                    restore_best_weights=True,
                                    verbose=1
                                )

# Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=2,  # Increased from 5
                    min_delta=0.0001,
                    verbose=1
                )

# Cosine annealing schedule (Improvement #5)
def cosine_annealing(epoch, lr):
    """Cosine annealing learning rate schedule."""
    import math
    # Fix:
    total_epochs = cfg.get("epochs", 50)
    initial_lr = cfg.get("learning_rate", 1e-4)
    min_lr = 1e-7
    
    if epoch < 5:  # Warmup phase
        return initial_lr * (epoch + 1) / 5
    else:
        progress = (epoch - 5) / (epochs - 5)
        return min_lr + (initial_lr - min_lr) * 0.5 * (1 + math.cos(math.pi * progress))

callbacks_list = [checkpoint, early_stopping, reduce_lr]

if cfg.get("use_lr_schedule", False):
    lr_scheduler = LearningRateScheduler(cosine_annealing, verbose=1)
    callbacks_list.append(lr_scheduler)
    print("✅ Using cosine annealing learning rate schedule")
else:
    callbacks_list.append(reduce_lr)
    print("✅ Using ReduceLROnPlateau")

print(f"✅ Callbacks configured: {len(callbacks_list)} callbacks")

✅ Using cosine annealing learning rate schedule
✅ Callbacks configured: 4 callbacks


In [13]:
print("cfg picture_size:", cfg.get("picture_size"))
print("cfg color_mode:", cfg.get("color_mode"))
print("train_set.image_shape:", getattr(train_set, "image_shape", None))
print("validation_set.image_shape:", getattr(validation_set, "image_shape", None))
print("model.input_shape:", getattr(model, "input_shape", None))

cfg picture_size: 96
cfg color_mode: rgb
train_set.image_shape: (96, 96, 3)
validation_set.image_shape: (96, 96, 3)
model.input_shape: (None, 96, 96, 3)


In [14]:
# =======================================================================
# Model Training
# =======================================================================

print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"Target: {epochs} epochs with early stopping")
print(f"Batch size: {batch_size}")
print(f"Learning rate: {learning_rate}")
print(f"Augmentation: {cfg['aug_level']}")
print(f"Backbone: {cfg['backbone']}")
print(f"Class weights: {'Enabled' if class_weight_dict else 'Disabled'}")
print("="*70)

# Full head training (frozen backbone)
history_head = model.fit(
    train_set,
    epochs=cfg["head_epochs"],  # e.g. 10
    validation_data=validation_set,
    callbacks=callbacks_list,
    verbose=1,
    class_weight=class_weight_dict
)
                        
                    
# Fine-tuning stage
if cfg.get("fine_tune", False) and cfg.get("fine_tune_epochs", 0) > 0 and base_model is not None:
    print("🔧 Starting fine-tuning...")
    
    # Unfreeze top layers
    N = cfg.get("fine_tune_unfreeze_layers", 30)
    for layer in base_model.layers[:-N]:
        layer.trainable = False
    for layer in base_model.layers[-N:]:
        layer.trainable = True

    # Recompile with lower LR
    ft_lr = cfg.get("learning_rate", 1e-4) * 0.1
    model.compile(optimizer=Adam(learning_rate=ft_lr), loss='categorical_crossentropy', metrics=['accuracy'])

    # Continue training
    history_ft = model.fit(
        train_set,
        epochs=cfg["head_epochs"] + cfg["fine_tune_epochs"],
        initial_epoch=cfg["head_epochs"],
        validation_data=validation_set,
        callbacks=callbacks_list,
        verbose=1,
        class_weight=class_weight_dict,
    )


🚀 STARTING TRAINING
Target: 5 epochs with early stopping
Batch size: 64
Learning rate: 0.0001
Augmentation: light
Backbone: mobilenet
Class weights: Enabled


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: LearningRateScheduler setting learning rate to 2e-05.
Epoch 1/10


I0000 00:00:1762914244.761577     106 service.cc:148] XLA service 0x7d683c010020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762914244.762765     106 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1762914244.762784     106 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1762914245.777780     106 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/451 ━━━━━━━━━━━━━━━━━━━━ 1:36:40 13s/step - accuracy: 0.0469 - loss: 3.1540

I0000 00:00:1762914251.672108     106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step - accuracy: 0.1654 - loss: 2.3981
Epoch 1: val_accuracy improved from -inf to 0.31588, saving model to best_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 255s 538ms/step - accuracy: 0.1655 - loss: 2.3978 - val_accuracy: 0.3159 - val_loss: 1.7819 - learning_rate: 2.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 4e-05.
Epoch 2/10
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.2560 - loss: 1.9742
Epoch 2: val_accuracy improved from 0.31588 to 0.34758, saving model to best_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 102s 227ms/step - accuracy: 0.2560 - loss: 1.9741 - val_accuracy: 0.3476 - val_loss: 1.7024 - learning_rate: 4.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 6.000000000000001e-05.
Epoch 3/10
451/451 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.2977 - loss: 1.8138
Epoch 3: val_accuracy improved from 0.34758 to 0.36824, saving model to best_model.keras
451/451 ━━━━━━━━━━━━━━━━━━━━ 102s 226ms/s

ZeroDivisionError: division by zero

In [ ]:
# =======================================================================
# Training History Visualization
# =======================================================================

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Loss plot
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Final evaluation
print("\n" + "="*70)
print("📊 FINAL EVALUATION")
print("="*70)
train_loss, train_acc = model.evaluate(train_set, verbose=0)
val_loss, val_acc = model.evaluate(validation_set, verbose=0)

print(f"Training Accuracy:   {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"Training Loss:       {train_loss:.4f}")
print(f"Validation Loss:     {val_loss:.4f}")
print("="*70)

In [ ]:
# =======================================================================
# Detailed Performance Analysis
# =======================================================================

from keras.models import load_model

# Load best model
my_model = load_model('best_model.keras', compile=False)

# Get predictions
print("Generating predictions on validation set...")
validation_set.reset()
predictions = my_model.predict(validation_set, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_set.classes

# Calculate metrics
accuracy = np.mean(predicted_classes == true_classes)
cm = confusion_matrix(true_classes, predicted_classes)

print(f"\n✅ Best Model Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

In [ ]:
# =======================================================================
# Enhanced Performance Visualizations
# =======================================================================

# 1. Confusion Matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Count'},
            linewidths=0.5,
            linecolor='gray')
plt.title('Confusion Matrix - Emotion Recognition', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
plt.ylabel('True Emotion', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Normalized Confusion Matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=class_labels,
            yticklabels=class_labels,
            cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix (%)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
plt.ylabel('True Emotion', fontsize=13, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

# 3. Classification Report
print("\n" + "="*70)
print("📈 DETAILED CLASSIFICATION METRICS")
print("="*70)
print(classification_report(true_classes, predicted_classes, 
                           target_names=class_labels,
                           digits=4))

# 4. Per-Class Metrics Bar Chart
precision, recall, f1, support = precision_recall_fscore_support(
    true_classes, predicted_classes, labels=range(7)
)

fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(class_labels))
width = 0.25

bars1 = ax.bar(x - width, precision, width, label='Precision', color='#3498db')
bars2 = ax.bar(x, recall, width, label='Recall', color='#2ecc71')
bars3 = ax.bar(x + width, f1, width, label='F1-Score', color='#e74c3c')

ax.set_xlabel('Emotion', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Per-Emotion Performance Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.1])

# Add value labels on bars
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('per_class_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. Summary Table
summary_df = pd.DataFrame({
    'Emotion': class_labels,
    'Precision': [f'{p:.2%}' for p in precision],
    'Recall': [f'{r:.2%}' for r in recall],
    'F1-Score': [f'{f:.2%}' for f in f1],
    'Support': support.astype(int)
})

print("\n" + "="*70)
print("📊 PERFORMANCE SUMMARY TABLE")
print("="*70)
print(summary_df.to_string(index=False))
print("="*70)

# 6. Misclassification Analysis
misclassified = cm.copy()
np.fill_diagonal(misclassified, 0)

top_confusions = []
for i in range(7):
    for j in range(7):
        if i != j:
            top_confusions.append((class_labels[i], class_labels[j], misclassified[i, j]))

top_confusions.sort(key=lambda x: x[2], reverse=True)

print("\n" + "="*70)
print("⚠️  TOP 5 MISCLASSIFICATION PAIRS")
print("="*70)
for true_label, pred_label, count in top_confusions[:5]:
    print(f"{true_label:>10} → {pred_label:<10} : {count:>4} times")
print("="*70)

In [ ]:
# =======================================================================
# Save Model with Timestamp
# =======================================================================

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"emotion_recognition_{timestamp}.keras"

if is_on_kaggle():
    output_path = "/kaggle/working/"+timestamp+"/"
else:
    output_path = "outputs/"+timestamp+"/"
    
try:
    os.makedirs(output_path, exist_ok=True)
    full_path = Path(output_path) / model_name
    model.save(str(full_path))
except Exception as e:
    print(f"Error saving model: {e}")

print(f"\n💾 Model saved: {full_path}")
print(f"📊 Final Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")

In [ ]:
# =======================================================================
# Ensemble Training (Optional)
# =======================================================================

if cfg.get("use_ensemble", False):
    print("\n" + "="*70)
    print("🎯 TRAINING ENSEMBLE MODELS")
    print("="*70)
    
    n_models = cfg.get("n_ensemble_models", 3)
    ensemble_models = []
    
    for i in range(n_models):
        print(f"\n{'='*70}")
        print(f"Training Ensemble Model {i+1}/{n_models}")
        print(f"{'='*70}")
        
        # Vary hyperparameters slightly
        cfg_copy = cfg.copy()
        cfg_copy["learning_rate"] = cfg["learning_rate"] * (0.8 + 0.4 * np.random.random())
        cfg_copy["dropout_rate"] = 0.3 + 0.2 * np.random.random()
        
        # Build model
        if cfg["backbone"] in ["mobilenet", "efficientnet"]:
            ensemble_model, _ = build_transfer_learning_model(cfg_copy)
        else:
            ensemble_model, _ = build_improved_cnn(cfg_copy)
        
        # Train
        history_ens = ensemble_model.fit(
            train_set,
            validation_data=validation_set,
            epochs=30,
            callbacks=[early_stopping, reduce_lr],
            class_weight=class_weight_dict,
            verbose=1
        )
        
        ensemble_models.append(ensemble_model)
        
        # Save
        ensemble_model.save(f"ensemble_model_{i+1}.keras")
    
    # Ensemble prediction
    print("\n" + "="*70)
    print("🎯 ENSEMBLE PREDICTION")
    print("="*70)
    
    validation_set.reset()
    ensemble_predictions = []
    
    for i, ens_model in enumerate(ensemble_models):
        print(f"Getting predictions from model {i+1}...")
        pred = ens_model.predict(validation_set, verbose=0)
        ensemble_predictions.append(pred)
    
    # Average predictions
    avg_predictions = np.mean(ensemble_predictions, axis=0)
    ensemble_predicted_classes = np.argmax(avg_predictions, axis=1)
    
    # Evaluate ensemble
    ensemble_accuracy = np.mean(ensemble_predicted_classes == true_classes)
    
    print(f"\n✅ Ensemble Accuracy: {ensemble_accuracy:.4f} ({ensemble_accuracy*100:.2f}%)")
    print(f"📈 Improvement over single model: {(ensemble_accuracy - accuracy)*100:.2f}%")
else:
    print("\n⚠️  Ensemble training disabled (set 'use_ensemble': True to enable)")

In [ ]:
# =======================================================================
# Sample Predictions Visualization
# =======================================================================
import random

# Get a random batch from validation set
random_batch_idx = random.randint(0, len(validation_set) - 1)
sample_images, sample_labels = validation_set[random_batch_idx]

# Predict
sample_preds = my_model.predict(sample_images, verbose=0)
sample_pred_classes = np.argmax(sample_preds, axis=1)
sample_true_classes = np.argmax(sample_labels, axis=1)

# Randomly select 16 indices from the batch
num_samples = min(16, len(sample_images))
random_indices = random.sample(range(len(sample_images)), num_samples)

# Visualize predictions
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for i, idx in enumerate(random_indices):
    axes[i].imshow(sample_images[idx].squeeze(), cmap='gray')
    
    true_label = class_labels[sample_true_classes[idx]]
    pred_label = class_labels[sample_pred_classes[idx]]
    confidence = sample_preds[idx][sample_pred_classes[idx]] * 100

    # Color: green if correct, red if wrong
    color = 'green' if sample_true_classes[idx] == sample_pred_classes[idx] else 'red'
    
    axes[i].set_title(f'True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)', 
                     color=color, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('Sample Predictions on Validation Set', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("Global policy:", mixed_precision.global_policy())
print("Model dtype policy:", model.dtype_policy)   # or model.input_dtype / model.output_dtype

In [ ]:
model.saved = True